# Cryptography and Information Security 
## Symmetric Block Cipher assignment

- Fathi Al Adha Hylmi (22/492195/PA/21008)
- Louis Widi Anandaputra(22/492218/PA/21090)

We well be constructing a symmetric block cipher for a text message. This method would also work with other data, but we will be implementing it on encrypting strings using the CBC structure (for more than 1 block data) in a feistel structure which uses a finite field arithmetic (addition operation) for a Galois Field(GF).

1 block wold contain 2^32 data and the finite field arithmetic would use a GF(2^16). The feistel structure rounds can be set as a parameter but we will be using a 16 rounds feistel structure. We also define several functions:

- initialGenKey(length) --> intializing keys for both the key and initial vector (IV). Taking an integer parameter length (bytes)
- subGenKey(key) --> creating subkeys for the feistel cipher encryption process. Taking an integer parameter (key)
- reverseSubGenKey(last_key) --> reversing back the subkeys to the original key. Taking an integer parameter (last_key)
- GFaddition(plain, key) --> Finite Field addition on GF(2^16). Taking integer parameters (plain) and (key)
<br><br>
- encrypt_messageCBC(blockList, key, iv, len_of_block, round) --> CBC structure encryption for all blocks. Taking string array parameter (blockList), bytearray parameters (key) and (iv), and integer parameters (len_of_block) and (round)
- encrypt(plain_block, key) --> initial encryption with IV. Taking string parameter (plain_block) and bytearray parameter (key)
- feistelEncrypt(plain, key, round) --> feistel structure block encryption. Taking bytearray parameter (plain) and (key), and taking integer parameter (round)
<br><br>
- decrypt_cipherCBC(list_cipher, key, iv, round) --> CBC structure decryption for all blocks. Taking bytearray parameters (list_cipher) and (iv), and taking integer parameters (key, round)
- feistelDecrypt(cipher, key_int, round) --> feistel structure block decryption. Taking bytearray parameter (list) and integer parameters (key_int) and (round)
- decrypt(cipher, key) --> decryption with IV
<br><br>
- breakMessage(message, len_of_blocks) --> breaking data into blocks. Taking string parameter (message) and taking integer parameters (len_of_blocks)

### reference for code: https://www.youtube.com/watch?v=wwTsRONdAaw
<br><br>
<img src = 'https://ctf-wiki.mahaloz.re/crypto/blockcipher/mode/figure/cbc_encryption.png'> <img src = 'https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Feistel_cipher_diagram_en.svg/300px-Feistel_cipher_diagram_en.svg.png'>

## Hashing and HMAC
We will be using 

## Keys and Finite Field Arithmetic

In [1]:
from os import urandom

# Initial key generator
def initialGenKey(length):
    return bytearray(urandom(length))
    
def subGenKey(key):
    # perform an addition by 1 and XOR by 1 and 0
    value = (key + 1) ^ 0x10
    # reset the value to 0 if it exceeds the maximum limit
    shifted_key = value & ((1 << 16) - 1)  # 16 bit limit
    print(f"Initial key: {key}; new key: {shifted_key}")
    return shifted_key
    
def reverseSubGenKey(last_key):
    # perform the inverse XOR operation to recover the value before XOR
    value = (last_key ^ 0x10)-1
    # reset the value to 0 if it exceeds the maximum limit
    key = value & ((1 << 16) - 1)# 16 bit limit
    print(f"Initial key: {last_key}; new key: {key}")
    return key
    
# we will be using a 2^16 finite field addition operation, which will be using XOR operation
def GFaddition(plain, key):
    value = plain ^ key
    value = value & ((1 << 16) - 1)
    return value

## Encryption

In [2]:
def encrypt_messageCBC(blockList, key, iv, len_of_block, round):
    list_of_ciphers = []
    for i in range(len(blockList)):
        print (f"\nBlock {i}:\n{blockList[i]}\n")
        # performing initial encryption with initial vector
        cipher1 = encrypt(blockList[i], iv)
        # performing the feistel cipher encryption
        cipher2, key_last = feistelEncrypt(cipher1, key, round)
        list_of_ciphers.append(cipher2)
        iv = cipher2
    return list_of_ciphers, key_last

In [3]:
def encrypt(plain_block, key):
    return bytearray([ord(plain_block[i]) ^ key[i] for i in range(len(plain_block))])

In [4]:
def feistelEncrypt(plain, key, round):
    # Changing key and plaintext into integers for encryption process
    key_int = int.from_bytes(key, byteorder='big')
    plain = int.from_bytes(plain, byteorder='big')

    # Splitting the bits to half (16 - 16)
    left = plain >> 16
    right = plain & 0xffff
    print(f"Initial left partition: {left}\nInitial right partition: {right}\n")

    # feistel structure
    for k in range ( round):  #rounds
        temp = GFaddition(right, key_int) # Using the Galois Field encryption
        left = left ^ temp
        print(f"Round {k+1}:Round {k+1}:\n right partition(after applied F, Key:{key_int}): {right}\n left partition(after XOR with right, right:{right}): {left}")

        # generating new subkey
        key_int = subGenKey(key_int)

        #switching the left and right data
        temp = right
        right = left
        left = temp
        print(f"Round {k+1} switching results:\n left partition: {left}\n right partition: {right}\n")

    # final switch for the left and right data   
    temp = right
    right = left
    left = temp    
    print(f"Final left partition: {left}\nInitial right partition: {right}\n")
    
    key = key_int
    # making sure the length is still 2^16
    encrypted_block = (left <<16)|right
    # changing back into bytes
    encrypted_block = encrypted_block.to_bytes((encrypted_block.bit_length() + 7) // 8, byteorder='big')
    return encrypted_block, key

## Decryption

In [5]:
def decrypt_cipherCBC(list_cipher, key, iv, round):
    list_of_decrypted_blocks = []
    temp, key_last = feistelDecrypt(list_cipher[0], key, round)
    plain = decrypt(temp, iv)
    list_of_decrypted_blocks.append(plain)
    for i in range(1, len(list_cipher)):
        temp, key_last = feistelDecrypt(list_cipher[i], key, round)
        plain = decrypt(temp, list_cipher[i-1])
        list_of_decrypted_blocks.append(plain)
    return list_of_decrypted_blocks


In [6]:
def feistelDecrypt(cipher, key_int, round):
    cipher = int.from_bytes(cipher, byteorder='big')
    key_int = reverseSubGenKey(key_int)
    left = cipher >> 16
    right = cipher & 0xffff
    print(f"Initial left partition: {left}\nInitial right partition: {right}\n")
  
    for k in range (round):  #rounds
        temp = GFaddition(right, key_int)
        left = left ^ temp
        
        
        print(f"Round {k+1}:\n right partition(after applied F, Key:{key_int}): {right}\n left partition(after XOR with right, right:{right}): {left}")
        
        key_int = reverseSubGenKey(key_int)
        
        temp = right
        right = left
        left = temp
        print(f"Round {k+1} switching results:\n left partition: {left}\n right partition: {right}\n")

    temp = right
    right = left
    left = temp
    print(f"Final left partition: {left}\nInitial right partition: {right}\n")
    
    key = key_int
    decrypted_block = (left <<16)|right
    decrypted_block = decrypted_block.to_bytes((decrypted_block.bit_length() + 7) // 8, byteorder='big')
    return decrypted_block, key

In [7]:
def decrypt(cipher, key):
    return [chr(cipher[i] ^ key[i]) for i in range(len(cipher))]

## Block Creation

In [8]:
def breakMessage(message, len_of_blocks):
    list_of_blocks = []
    for i in range (0, len(message), len_of_blocks):
        block = message[i:i+len_of_block]
        if(len(block) == len_of_blocks):
            list_of_blocks.append(block)
        else:
            c = len_of_blocks - len(block)
            for i in range(c):
                block = block + " "
            list_of_blocks.append(block)
    return list_of_blocks

def breakMessage_hash(message, len_of_blocks):
    if isinstance(message, str):
        message = message.encode()
    list_of_blocks = []
    for i in range(0, len(message), len_of_blocks):
        block = message[i:i + len_of_blocks]
        if len(block) == len_of_blocks:
            list_of_blocks.append(block)
        else:
            c = len_of_blocks - len(block)
            block += b' ' * c
            list_of_blocks.append(block)
    return list_of_blocks
                    

## Hash and Message Authentication

In [9]:
def bitwise_rotate_left(val, r_bits, max_bits): 
    """
    val : value to be rotated
    r_bits = number of rotation step
    max_bits = maximum bit rotations
    """
    return (val << r_bits%max_bits) & (2**max_bits-1) | ((val & (2**max_bits-1)) >> (max_bits-(r_bits%max_bits)))

def hashFunction(message, len_of_block, key, iv, hmac_outer = False):
    """
    message : plaintext (string if not for outer hash in hmac, block array for outer hash in hmac)
    len_of_block: n bytes of block length
    key : key for XOR with Initial Value
    IV: Initial Value for XOR with key
    """
    blockList = message
    if hmac_outer == False:
        blockList = breakMessage_hash(message, len_of_block)
        blockList = [int.from_bytes(i, byteorder = 'big') for i in blockList]
    
    # Start of Rotated XOR
    block = iv^key
    for m in blockList:
        
        temp = bitwise_rotate_left(block, 1, (8*(len_of_block)))
        block = m ^ temp 
    
    return block

def HMAC(message, key, len_of_block, iv):
    """
    message : plaintext (string if not for outer hash in hmac, block array for outer hash in hmac)
    len_of_block: n bytes of block length
    key : key for operation with outer and inner padding in hash function
    IV: Initial Value for XOR with key in hash function
    """
    key = int.from_bytes(key, byteorder = 'big')
    iv = int.from_bytes(iv, byteorder = 'big')

        
    o_key_pad = key ^ 0x5C
    i_key_pad = key ^ 0x36

    inner_hash = hashFunction(message, len_of_block, i_key_pad, iv) # NOTE for testing, make the hmac_outer = True
    print("inner_hash:",bin(inner_hash))
    inner_hash = [inner_hash]
    hmac_output = hashFunction(inner_hash, len_of_block, o_key_pad, iv, hmac_outer = True)

    return hmac_output

## Demo

In [39]:
# Initialization
text = "Hello world! This is Louis and Hylmi."
len_of_block = 4
len_of_feistel = 2
len_of_block_hmac = 2
round = 16
blockList = breakMessage(text, len_of_block)

In [11]:
key = initialGenKey(len_of_feistel)
iv = initialGenKey(len_of_block)

key_hmac = initialGenKey(len_of_block_hmac)
iv_hmac = initialGenKey(len_of_block_hmac)


In [34]:
print(f"Key: {key}\nIV: {iv}\nKey HMAC: {key_hmac}\nIV HMAC: {iv_hmac}")

Key: bytearray(b'1n')
IV: bytearray(b'\xf5\x14\x0c\x1b')
Key HMAC: bytearray(b'\xd8J')
IV HMAC: bytearray(b'\xd3r')


In [42]:
key = bytearray(b'1n')
iv =  bytearray(b'\xf5\x14\x0c\x1b')
key_hmac = bytearray(b'\xd8J')
iv_hmac = bytearray(b'\xd3r')

In [43]:
%%time
# encryption
cipher_list, key_last_used= encrypt_messageCBC(blockList, key, iv, len_of_block, round = round)


Block 0:
Hell

Initial left partition: 48497
Initial right partition: 24695

Round 1:Round 1:
 right partition(after applied F, Key:12654): 24695
 left partition(after XOR with right, right:24695): 60520
Initial key: 12654; new key: 12671
Round 1 switching results:
 left partition: 24695
 right partition: 60520

Round 2:Round 2:
 right partition(after applied F, Key:12671): 60520
 left partition(after XOR with right, right:60520): 48480
Initial key: 12671; new key: 12688
Round 2 switching results:
 left partition: 60520
 right partition: 48480

Round 3:Round 3:
 right partition(after applied F, Key:12688): 48480
 left partition(after XOR with right, right:48480): 24728
Initial key: 12688; new key: 12673
Round 3 switching results:
 left partition: 48480
 right partition: 24728

Round 4:Round 4:
 right partition(after applied F, Key:12673): 24728
 left partition(after XOR with right, right:24728): 60537
Initial key: 12673; new key: 12690
Round 4 switching results:
 left partition: 24728

In [44]:
%%time
# decryption
decrypted_list = decrypt_cipherCBC(cipher_list, key_last_used, iv, round)

Initial key: 12702; new key: 12685
Initial left partition: 60541
Initial right partition: 24732

Round 1:
 right partition(after applied F, Key:12685): 24732
 left partition(after XOR with right, right:24732): 48492
Initial key: 12685; new key: 12700
Round 1 switching results:
 left partition: 24732
 right partition: 48492

Round 2:
 right partition(after applied F, Key:12700): 48492
 left partition(after XOR with right, right:48492): 60524
Initial key: 12700; new key: 12683
Round 2 switching results:
 left partition: 48492
 right partition: 60524

Round 3:
 right partition(after applied F, Key:12683): 60524
 left partition(after XOR with right, right:60524): 24715
Initial key: 12683; new key: 12698
Round 3 switching results:
 left partition: 60524
 right partition: 24715

Round 4:
 right partition(after applied F, Key:12698): 24715
 left partition(after XOR with right, right:24715): 48509
Initial key: 12698; new key: 12681
Round 4 switching results:
 left partition: 24715
 right parti

In [45]:
HMAC_Value = HMAC(text, key_hmac, len_of_block_hmac, iv_hmac)
bin(HMAC_Value)

inner_hash: 0b111100011000000


'0b110111000001000'

In [46]:
cipher_list_appended = cipher_list.copy()
cipher_list_appended.append([HMAC_Value])
cipher_list_appended

[b'\xec}`\x9c',
 b'\xa5\xd5\x17\x18',
 b'\x95\xfbs\xd2',
 b'\x9fo\x1bP',
 b'\xaf\x17r\xc8',
 b'\xa3\x9d\x1dV',
 b'\xc6\xa2=\xdc',
 b'\x84)\x1d\x7f',
 b'\xbc(p\xfd',
 b'\xf3\xaeP6',
 [28168]]

### Results
#### Plain Text

In [47]:
# plaintext
for i in blockList:
    print ([(i[k]) for k in range(len(i))])
    

['H', 'e', 'l', 'l']
['o', ' ', 'w', 'o']
['r', 'l', 'd', '!']
[' ', 'T', 'h', 'i']
['s', ' ', 'i', 's']
[' ', 'L', 'o', 'u']
['i', 's', ' ', 'a']
['n', 'd', ' ', 'H']
['y', 'l', 'm', 'i']
['.', ' ', ' ', ' ']


#### Original Cipher Text

In [48]:
# ciphertext
for i in cipher_list:
    print ([chr(i[k]) for k in range(len(i))])


['ì', '}', '`', '\x9c']
['¥', 'Õ', '\x17', '\x18']
['\x95', 'û', 's', 'Ò']
['\x9f', 'o', '\x1b', 'P']
['¯', '\x17', 'r', 'È']
['£', '\x9d', '\x1d', 'V']
['Æ', '¢', '=', 'Ü']
['\x84', ')', '\x1d', '\x7f']
['¼', '(', 'p', 'ý']
['ó', '®', 'P', '6']


#### HMAC Appended Cipher Text

In [49]:
# ciphertext
for i in cipher_list_appended:
    print ([chr(i[k]) for k in range(len(i))])

['ì', '}', '`', '\x9c']
['¥', 'Õ', '\x17', '\x18']
['\x95', 'û', 's', 'Ò']
['\x9f', 'o', '\x1b', 'P']
['¯', '\x17', 'r', 'È']
['£', '\x9d', '\x1d', 'V']
['Æ', '¢', '=', 'Ü']
['\x84', ')', '\x1d', '\x7f']
['¼', '(', 'p', 'ý']
['ó', '®', 'P', '6']
['済']


#### Decrypted Back to Plain Text

In [30]:
# plaintext
for i in decrypted_list:
    print ([(i[k]) for k in range(len(i))])

['H', 'e', 'l', 'l']
['o', ',', ' ', 'W']
['o', 'r', 'l', 'd']
['!', ' ', ' ', ' ']


© 2024 - Fathi Al Adha Hylmi & Louis Widi Anandaputra